In [ ]:
from docx import Document
from docx.shared import Inches
from PIL import Image
import math
import tkinter as tk
from tkinter import filedialog

# Predefined page sizes in inches (W,H)
PAGE_SIZES = {
    "A3": (11.7, 16.5),
    "A4": (8.27, 11.69),
    "A5": (5.83, 8.27)
}

def main():
    # 1. Ask for filename
    filename = input("Enter output filename (with .docx): ")
    
    # 2. Ask for number of images
    n = int(input("How many pictures to add? "))

    # 3. Ask for page size
    print("Choose page size (A3/A4/A5):")
    sheet = input().upper()
    page_width, page_height = PAGE_SIZES.get(sheet, PAGE_SIZES["A4"])  # default A4

    # 4. File upload dialog
    print("📂 Select your images...")
    root = tk.Tk()
    root.withdraw()  # Hide main tkinter window
    image_paths = filedialog.askopenfilenames(
        title="Select Images",
        filetypes=[("Image Files", "*.jpg *.jpeg *.png *.bmp *.tif *.tiff")]
    )
    image_paths = list(image_paths)

    if len(image_paths) != n:
        print(f"⚠️ You selected {len(image_paths)} files but said {n}. Adjusting to selected files.")
        n = len(image_paths)

    # 5. Create document
    doc = Document()
    section = doc.sections[0]
    section.page_width = Inches(page_width)
    section.page_height = Inches(page_height)

    # Auto grid size (square-ish)
    cols = math.ceil(math.sqrt(n))
    rows = math.ceil(n / cols)

    table = doc.add_table(rows=rows, cols=cols)
    table.autofit = True

    idx = 0
    for r in range(rows):
        row_cells = table.rows[r].cells
        for c in range(cols):
            if idx < n:
                # Get cell dimensions
                cell_width = page_width / cols
                cell_height = page_height / rows

                # Open image to check size
                img = Image.open(image_paths[idx])
                iw, ih = img.size
                aspect = iw / ih

                # Fit inside cell
                max_w = Inches(cell_width)
                max_h = Inches(cell_height)
                if iw > ih:   # landscape
                    doc_width = min(max_w, max_h * aspect)
                    row_cells[c].paragraphs[0].add_run().add_picture(image_paths[idx], width=doc_width)
                else:         # portrait
                    doc_height = min(max_h, max_w / aspect)
                    row_cells[c].paragraphs[0].add_run().add_picture(image_paths[idx], height=doc_height)

                idx += 1

    # 6. Save file
    doc.save(filename)
    print(f"✅ File saved as {filename}")

if __name__ == "__main__":
    main()
